In [2]:
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
import keras
from keras.layers import LSTM 
from keras.models import Sequential
from transformers import pipeline
import riiideducation

ModuleNotFoundError: No module named 'competition'

In [ ]:
def percentage_correct(df):
    return len(df[df['answered_correctly']==1])/len(df[df['answered_correctly']!=-1])

In [ ]:
print(pipeline('sentiment-analysis')('we love to hate you'))

In [ ]:
lectures = pd.read_csv('lectures.csv')
lectures.head()

In [ ]:
train = pd.read_pickle('riiid_train.pkl.gzip')
train.head()

In [ ]:
questions = pd.read_csv('questions.csv')
questions.head()

In [ ]:
len(train)

In [ ]:
print(len(train[train['user_answer']!=-1]))

In [ ]:
print(len(train[train['answered_correctly']==1])/len(train[train['answered_correctly']!=-1]))

In [ ]:
sample_user = train[train['user_id'] == 115]
sample_user.head()

In [ ]:
plt.figure(figsize=(20,5),edgecolor='black')
plt.plot(sample_user['answered_correctly'])
# plt.plot(sample_user['prior_question_elapsed_time']/60000)
# plt.plot(sample_user['prior_question_had_explanation'])

In [ ]:
users = np.unique(train['user_id'])
print("Number of unique users == " + str(len(users)))

In [ ]:
unique_lecture_tags = lectures.tag.unique()
questions[questions.tags.isna()]

In [ ]:
questions['tags'] = questions['tags'].astype(str)

tags = [x.split() for x in questions[questions.tags != "nan"].tags.values]
tags = [item for elem in tags for item in elem]
tags = set(tags)
tags = list(tags)
print(f'There are {len(tags)} different tags')

In [ ]:
cids = train.content_id.value_counts()[:50]

fig = plt.figure(figsize=(12,6))
ax = cids.plot.bar()
# plt.title("Thirty most used content id's")
plt.xticks(rotation=90)
plt.show()

In [ ]:
correct = train[train.answered_correctly != -1].answered_correctly.value_counts()

fig = plt.figure(figsize=(4,4))
plt.pie(correct,labels=['correct','incorrect'], autopct='%1.1f%%',colors=['green','maroon'])
plt.title("Questions answered correctly")
plt.xticks(rotation=0)
plt.show()

In [ ]:
users = train.groupby('user_id')

In [ ]:
percentage_correct_per_user = []
for key, value in users:
    percentage_correct_per_user.append(percentage_correct(value))
percentage_correct(train)

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
y = percentage_correct_per_user[:10]
x = np.unique(train['user_id'])[:10]
x = [str(s) for s in x]
ax.bar(x,y)
plt.show()


In [ ]:
questions.nunique()

In [ ]:
questions['tags'] = questions['tags'].astype(str)
tags = [x.split() for x in questions[questions.tags != "nan"].tags.values]
tags = [item for elem in tags for item in elem]
values, counts = np.unique(tags, return_counts=True)

In [ ]:
questions['tags'] = questions['tags'].astype(str)
tags = [x.split() for x in questions[questions.tags != "nan"].tags.values]
tags = [item for elem in tags for item in elem]
maps = pd.DataFrame(tags)
maps = maps.value_counts()
keys = [x for (x,) in maps.index.values]
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
y = maps.values[:20]
x = keys[:20]
ax.bar(x,y)
plt.show()

In [ ]:
temp = questions.query('tags.str.contains("92")', engine='python')

In [ ]:
temp.head()

In [ ]:
temp = train.query('answered_correctly != -1')

In [ ]:
merged_data = pd.merge(train[train['answered_correctly']!=-1],questions,left_on='content_id',right_on='question_id')

In [ ]:
merged_data.head()

In [ ]:
questions[questions['question_id']== 5692]

In [ ]:
merged_data.to_pickle('merged_questions.pkl')

In [ ]:
data = pd.read_pickle('merged_questions.pkl')

In [ ]:
def one_hot_encode_tags(data):
    data['tags'] = data['tags'].astype(str)
    temp_str = [x.split() for x in data[data.tags != "nan"].tags]
    data['tags'] = temp_str
    mlb = MultiLabelBinarizer(sparse_output=True)
    temp_data = data.join(
                pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(data.pop('tags')),
                index=data.index,
                columns=mlb.classes_))
    return temp_data            

In [ ]:
train.head()

In [ ]:
# Plot % of students whho answered a question correctly

qdata = data.groupby('content_id').sum()
qdata['answered_correctly']

In [ ]:
q = data.groupby('content_id').sum()/data.groupby('content_id').count()

In [ ]:
q.head()

In [ ]:
q['answered_correctly'] = q['answered_correctly']*100

In [ ]:
q.head()

In [ ]:
plt.bar(q['question_id'][-50:], q['answered_correctly'][-50:])

In [ ]:
q= q.sort_values('answered_correctly')


In [ ]:
c = data.groupby('content_id').count()

In [ ]:
c.head()
c.sort_values('q')

In [ ]:

fig = plt.figure(figsize=(20,5))
q['question_id'] = q['question_id'].astype(str)
q['answered_correctly'] = q['answered_correctly'].astype(int)
plt.bar(q['question_id'][:10], q['answered_correctly'][:10])
plt.bar(c['question_id'][:10], c['answered_correctly'][:10])


In [ ]:
data.head()